Notebook with code merging weather and air pollution datasets

In [1]:
import pandas as pd

In [2]:
station_pairs = pd.read_csv('final_pairs.csv')
station_pairs

,IMGWCode,AIRCode,Place,Distance,Unnamed: 4
0,353210285,MzOstrolKoscWSSE,Ostroleka,0.338655,NaN
1,349220690,PkLeskoWSSERynek,Lesko,0.348857,NaN
2,354170120,PmLebaRabkaE,Leba,0.478184,NaN
3,349200660,MpNoSaczWSSENawo17a,NowySacz,0.581714,NaN
4,354170125,PmLebMalcz16,Lebork,0.599580,NaN
...,...,...,...,...,...
57,354180155,PmGdaKacze02,Gdansk-Swibno,26.058689,NaN
58,354180135,Pm.00.s473m,Hel,27.721633,NaN
59,353150210,ZpGryficeWSSE,Resko,29.651610,NaN
60,353210280,WmMragowWSSE_Krolew,Mikolajki,32.654709,NaN


In [3]:
max_distance = 3
stations_mapping = {el[0]: el[1] for el in station_pairs.values if el[3] <= max_distance}
stations_mapping
stations_mapping2 = {el[1]: el[0] for el in station_pairs.values if el[3] <= max_distance}

In [309]:
[station for station in stations_mapping.keys()]

[353210285,
 349220690,
 354170120,
 349200660,
 354170125,
 350150510,
 352220385,
 349210670,
 353140200,
 353170235,
 352190360,
 354150100,
 349190625,
 354160105,
 354210185,
 354190160,
 351160415,
 354160115,
 350180540,
 353180250,
 351180455,
 349220695,
 351150400,
 351180435,
 350190550,
 350200575,
 350190560,
 351210488,
 350210585,
 354220195,
 351230497,
 353160230,
 349190600,
 353200270,
 353160215]

In [310]:
years = range(2008, 2021, 1)
parameters = {'temperature':'B00300S', 'wind': 'B00702A', 'precipitation_1':'B00606S', 'humidity': 'B00802A'}
dtypes={'station': 'string', 'temperature': float, 'wind': float, 'precipitation_24': float, 'precipitation_1': float, 'humidity': float}
data = pd.DataFrame()
for year in years:
    data = pd.DataFrame()
    for month in [str(el).rjust(2, '0') for el in range(1, 13, 1)]:
        for station in stations_mapping.keys():
            station_df = pd.DataFrame(columns=['station'])
            for param, code in parameters.items():
                df = pd.read_csv(f'pogoda/{year}/{code}_{year}_{month}.csv', decimal=',', sep=';', header=None, index_col=1, parse_dates=True, usecols=[0,2,3], names=["station", "date", param], dtype=dtypes)
                value_df = df.loc[df['station'] == str(station)]
                if not value_df.empty:
                    station_df = station_df.merge(value_df, how="outer", on="station", left_index=True, right_index=True)
            data = pd.concat([data, station_df])
    data.to_csv(f'pogoda/joined/{year}.csv')
    data.dropna().to_csv(f'pogoda/joined_dropped/{year}.csv')

In [7]:
data.dropna()

,station,temperature,precipitation_1,humidity,wind
2009-01-01 00:00:00,354170125,-7.7,0.0,95.0,0.0
2009-01-01 01:00:00,354170125,-6.2,0.0,96.0,0.0
2009-01-01 02:00:00,354170125,-4.6,0.0,96.0,1.1
2009-01-01 03:00:00,354170125,-4.1,0.0,95.0,1.3
2009-01-01 04:00:00,354170125,-3.3,0.0,96.0,1.5
...,...,...,...,...,...
2009-12-31 19:00:00,352190360,-2.9,0.0,73.0,4.4
2009-12-31 20:00:00,352190360,-2.8,0.0,70.0,4.3
2009-12-31 21:00:00,352190360,-2.9,0.0,72.0,4.1
2009-12-31 22:00:00,352190360,-3.0,0.0,73.0,3.8


SMOG

In [8]:
data = pd.read_excel('smog/2018/2018_PM25_1g.xlsx', header=0, skiprows=lambda x: x in [0,2,3,4, 5],index_col=0, parse_dates=True)

In [9]:
data

,DsJelGorOgin,DsWrocAlWisn,DsWrocWybCon,KpBydPlPozna,KpBydWarszaw,KpToruDziewu,KpWloclOkrze,LbLubObywate,LdLodzCzerni,LdLodzGdansk,...,SkKonsGranatMOB,SkPolaRuszcz,SlBielPartyz,SlKatoKossut,SlZlotPotLes,WmOlsPuszkin,WpKaliSawick,WpPoznDabrow,ZpSzczAndr01,ZpSzczPils02
2018-01-01 01:00:00,"43,0102","71,494","70,2","49,2716","69,6873","18,778",NaN,"30,9","16,0","68,349",...,NaN,"27,68",NaN,"76,8469","25,9266","41,3687",NaN,NaN,"26,5445","12,8035"
2018-01-01 02:00:00,"5,38425","7,89305","13,1522","36,236","42,3178",NaN,NaN,"54,8","83,0","39,999",...,NaN,"18,48",NaN,"39,3395","32,7522","25,8502",NaN,NaN,NaN,NaN
2018-01-01 03:00:00,"4,52882","12,5392","12,5","14,7514",NaN,NaN,NaN,"24,3","16,0","17,949",...,NaN,"15,59",NaN,"23,2554","37,6664","22,4599",NaN,NaN,"10,4332","32,0944"
2018-01-01 04:00:00,"8,68036","5,03196","8,85164","17,1654",NaN,"13,353",NaN,"15,1","14,0","12,699",...,NaN,"12,32",NaN,"22,3614","32,4625","20,1625",NaN,NaN,"28,1885","27,3274"
2018-01-01 05:00:00,"7,2","11,9136","7,29656","12,5788",NaN,NaN,NaN,"12,6","12,0","12,699",...,NaN,"10,03",NaN,"28,9092","27,135","18,3681",NaN,NaN,"40,1738","61,1957"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 20:00:00,"99,2267","43,6255","32,0664","32,95258","37,0","19,9155","49,0272","13,1","27,547","33,699",...,NaN,"9,05","37,0833","39,3415","16,6016","20,2928","19,1919","39,9397","15,9911","17,2711"
2018-12-31 21:00:00,"99,0422","39,6576","41,8542","24,06","21,5","27,353","37,1728","13,5","29,341","34,749",...,NaN,"12,41","44,4883","44,4603","22,982","19,2542","29,3164","42,7089","22,6865","18,7469"
2018-12-31 22:00:00,"126,327","31,3481","30,4876","15,28759","21,3","28,4905","34,2092","15,0","27,547","33,699",...,NaN,"13,85","36,596","48,8999","30,555","16,4678","33,0235","42,4264","16,6687","23,7844"
2018-12-31 23:00:00,"100,4","40,2615","30,8172","18,0515","20,2",NaN,"40,5188","11,5","27,547","27,399",...,NaN,"13,61","58,2189","42,6519","22,8511","17,3469","39,4939","47,3919","14,7215","20,0236"


In [314]:
for file in ['_NO2_1g', '_O3_1g', '_PM10_1g', '_PM25_1g', '_SO2_1g']:
    pollutant_frame = pd.DataFrame()
    for year in range(2008, 2021, 1):
        if (year < 2015 and file == '_PM25_1g'):
            name = f'smog/{year}/{year}_PM2.5_1g.xlsx'
        else: 
            name = f'smog/{year}/{year}{file}.xlsx'
        rows = [0,2,3,4,5] if year > 2015 else [1, 2]
        data = pd.read_excel(name, header=0, skiprows=lambda x: x in rows, index_col=0, parse_dates=True)
        pollutant_frame = pd.concat([pollutant_frame, data])
    pollutant_frame[[el for el in data.columns if el in stations_mapping.values()]].to_csv(f'smog/{file}.csv')

In [11]:
[el for el in data.columns if el in stations_mapping.values()]

['DsSniezkaObs',
 'LuZielKrotka',
 'MpZakopaSien',
 'PmLebaRabkaE',
 'PmLebMalcz16']

In [320]:
weather_df = pd.DataFrame()
for year in range(2008, 2021, 1):
    weather = pd.read_csv(f'pogoda/joined_dropped/{year}.csv', parse_dates=True, index_col=0)
    weather_df = pd.concat([weather_df, weather])
weather_df.set_index(['station'], inplace=True, append=True)


In [321]:
files = ['NO2', 'O3', 'PM10', 'PM25', 'SO2']
for file in files:
    pollutant = pd.read_csv(f'smog/_{file}_1g.csv', parse_dates=True, index_col=0)
    pollutant.index = pollutant.index.round('H')
    pollutant_df = pd.DataFrame(columns=['date', 'station', file])
    pollutant_df.set_index(['date', 'station'], inplace=True)
    for column in pollutant.columns:
      temp_df = pd.DataFrame(pollutant[column])
      temp_df = temp_df.assign(station=stations_mapping2[column])  
      temp_df.set_index(['station'], append=True, inplace=True)
      temp_df.columns = [file]
      pollutant_df = pd.concat([pollutant_df, temp_df])
    pollutant_df.dropna(inplace=True)
    weather_df = pd.merge(weather_df, pollutant_df, how='left', on=['date', 'station'])
weather_df.to_csv('weather_smog.csv')

In [336]:
for file in files:
   print(file, len(weather_df[[file]].dropna())) 

NO2 124930
O3 164105
PM10 105253
PM25 49582
SO2 49229
